# Test URL Download

In [2]:
from rook.wcrawl.event import events

In [3]:
%run ./jupyter-test-support.ipynb

In [6]:
%run ./create_container.ipynb

Created [container]
  * app_root
  * blob_root_directory
  * blob_store
  * environment_name
  * event_publisher
  * mongodb_client
  * mongodb_credentials
  * queue_receiver
  * queue_sender
  * rabbitmq_connection
  * rabbitmq_url
  * url_download_handler


In [7]:
publisher = container.event_publisher()

In [8]:
publisher.send_url_download('https://www.nytimes.com/')
publisher.send_url_download('https://www.yahoo.com/')

In [9]:
q_receiver = container.queue_receiver()
handler = container.url_download_handler()

## Try receiving
Right now this method will listen forever and needs to be stopped by sending an interupt.

In [10]:
q_receiver.receive(events.URL_DOWNLOAD, handler)

Request to download: {'url': 'https://www.nytimes.com/'}
Request to download: {'url': 'https://www.yahoo.com/'}


KeyboardInterrupt: 